In [21]:
!pip install requests beautifulsoup4 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://srv-nexus.sbs.gob.pe/repository/pypi_public/simple


In [24]:
#import beautifulsoup
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [25]:
#Paso 1: Web Scraping con BeautifulSoup

# Función para extraer noticias de una web de noticias (ejemplo de scraping)
def obtener_noticias(url):
    noticias = []
    response = requests.get(url)
    
    # Verifica que el acceso a la página es exitoso
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extraer titulares y contenido de artículos
        for link in soup.find_all('h2'):
            # Suponiendo que los titulares están en <h2> y el texto en <p>
            #titulo = article.find('h2').get_text(strip=True) if article.find('h2') else ''
            titulo = link.get_text(strip=True)
            #contenido = article.find('p').get_text(strip=True) if article.find('p') else ''
            
            # Limpiar texto si es necesario (quitar caracteres especiales)
            titulo = re.sub(r'\s+', ' ', titulo)
            #contenido = re.sub(r'\s+', ' ', contenido)
            
            #noticias.append({'titulo': titulo, 'contenido': contenido})
            noticias.append(titulo)
            
    return noticias

# URL de ejemplo (puedes ajustar a una web de noticias real)
url = 'https://gestion.pe/?ref=gesr'
noticias = obtener_noticias(url)

# Ver cuántas noticias se han extraído
print(f'Noticias extraídas: {len(noticias)}')


Noticias extraídas: 53


In [31]:
#Paso 2: Aplicar BM25 para buscar "fraudes electrónicos"


from sklearn.feature_extraction.text import TfidfVectorizer

# Función para aplicar BM25 sobre los títulos extraídos
def bm25_search(corpus, query, top_n=10, k1=1.5, b=0.75):
    # Vectorización de los documentos utilizando TF-IDF
    vectorizer = TfidfVectorizer(use_idf=True)
    X = vectorizer.fit_transform(corpus)
    
    # Extraer IDF (Inverse Document Frequency) de los términos
    idf = vectorizer.idf_
    tf = X.toarray()  # Term frequency
    
    # Longitud promedio de los documentos
    avgdl = np.mean([len(doc.split()) for doc in corpus])
    
    # Función para calcular el puntaje BM25
    def score(doc_index, query_terms):
        score = 0
        doc_length = len(corpus[doc_index].split())
        for term in query_terms:
            if term in vectorizer.vocabulary_:
                term_index = vectorizer.vocabulary_[term]
                term_freq = tf[doc_index][term_index]
                
                # Fórmula BM25
                idf_term = idf[term_index]
                numerator = term_freq * (k1 + 1)
                denominator = term_freq + k1 * (1 - b + b * (doc_length / avgdl))
                score += idf_term * (numerator / denominator)
        return score
    
    # Procesar la consulta de búsqueda
    query_terms = query.split()
    
    # Calcular puntajes BM25 para cada documento
    scores = [score(i, query_terms) for i in range(len(corpus))]
    
    # Obtener los top N documentos
    ranked_docs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    
    return ranked_docs, scores

# URL de la página web
url = 'https://gestion.pe/?ref=gesr'

# Etapa 1: Extraer los títulos de la página
titulos = obtener_noticias(url)

# Ver cuántos titulares se han extraído
print(f'Titulares extraídos: {len(titulos)}')

# Etapa 2: Aplicar BM25 para buscar noticias relevantes sobre "fraudes electrónicos"
query = "desgravamen"
top_docs, bm25_scores = bm25_search(titulos, query, top_n=10)

# Mostrar los resultados más relevantes (títulos)
for i in top_docs:
    print(f"Titulo: {titulos[i]}")
    print(f"Puntaje BM25: {bm25_scores[i]:.4f}")

Titulares extraídos: 53
Titulo: “Prácticas abusivas” con seguro de desgravamen serían sancionadas
Puntaje BM25: 2.7859
Titulo: MEF plantea “recortes prioritarios” para cumplir meta fiscal
Puntaje BM25: 0.0000
Titulo: Proyecto que busca darle fin al Reinfo sale la próxima semana, ¿qué se prepara?
Puntaje BM25: 0.0000
Titulo: Gasoducto al sur: TGP solicita 10 años más de periodo de concesión
Puntaje BM25: 0.0000
Titulo: Estos 20 distritos “ricos” contrarreloj para gastar más de S/ 2,000 millones
Puntaje BM25: 0.0000
Titulo: Midagri alista cambios a Agrobanco que agravarían su situación
Puntaje BM25: 0.0000
Titulo: Hay más trabajadores con educación universitaria que no pueden cubrir canasta mínima
Puntaje BM25: 0.0000
Titulo: Oficinas Clase A: los metros cuadrados que están en pausa y una renta que podría subir
Puntaje BM25: 0.0000
Titulo: Día de la Canción Criolla: estrategias de los restaurantes para subir las ventas
Puntaje BM25: 0.0000
Titulo: Economía bajo análisis: tres razones que